In [1]:
import pandas as pd
import os
import sys
# Add the parent directory to the Python path
parent_dir = os.path.abspath(os.path.join('..'))
sys.path.append(parent_dir)
# from travel_time_matrix import travel_time_matrix
from Metric_Calculation.MetricCalculation import MetricCalculation
parent_dir = os.path.abspath(os.path.join('../..'))
sys.path.append(parent_dir)
from data import data


### 1. Load the travel time matrix result generated using r5py and create Metric Calculation object
    

In [ ]:
# Before calculation
# Ensure the file paths are correct
travel_time_matrix_path = "../results/TTM_CT_hospitals_only_before.csv"
num_of_dest_output_path = "../results/TTM_CT_hospitals_before.csv"
thresholds = [30]
n_travel_time_output_path = "../results/first_hospitals_before.csv"

TTM_Metric = MetricCalculation(travel_time_matrix_path) 
TTM_Metric.filter_destinations( output_path = num_of_dest_output_path, threshold = thresholds )  # Filter the destinations to only include those within 30 minutes travel time
TTM_Metric.get_nth_travel_time(n  = 1, output_path= n_travel_time_output_path)

Generated ../results/TTM_CT_hospitals_before_threshold_30.csv
N-th travel time (N=1) saved to ../results/first_hospitals_before.csv


,from_id,to_id,travel_time
0,5350001.00,773891,73.0
1,5350002.00,54248,100.0
2,5350003.00,10757635,17.0
3,5350004.00,8168605,16.0
4,5350005.00,8168605,13.0
...,...,...,...
578,5350378.24,4616608,40.0
579,5350378.25,6362828,47.0
580,5350378.26,6362828,42.0
581,5350378.27,6362828,84.0


In [ ]:
# After calculation
travel_time_matrix_path = "../results/TTM_CT_hospitals_only_after.csv"
num_of_dest_output_path = "../results/TTM_CT_hospitals_after.csv"
thresholds = [30]
n_travel_time_output_path = "../results/first_hospitals_after.csv"

TTM_Metric = MetricCalculation(travel_time_matrix_path) 
TTM_Metric.filter_destinations( output_path = num_of_dest_output_path, threshold = thresholds )  # Filter the destinations to only include those within 30 minutes travel time
TTM_Metric.get_nth_travel_time(n  = 1, output_path= n_travel_time_output_path)

Generated ../results/TTM_CT_hospitals_after_threshold_30.csv
N-th travel time (N=1) saved to ../results/first_hospitals_after.csv


,from_id,to_id,travel_time
0,5350001.00,773891,73.0
1,5350002.00,54248,100.0
2,5350003.00,10757635,17.0
3,5350004.00,8168605,16.0
4,5350005.00,8168605,13.0
...,...,...,...
578,5350378.24,4616608,40.0
579,5350378.25,6362828,47.0
580,5350378.26,6362828,42.0
581,5350378.27,6362828,84.0


### 2. Combine before and after calculation to one table for visualization

In [ ]:
# Number of destinations within 30 minutes before and after

threshold = 30

before_csv_path = f"../results/TTM_CT_hospitals_before_threshold_{threshold}.csv"
after_csv_path = f"../results/TTM_CT_hospitals_after_threshold_{threshold}.csv"

CTUID_source = "../results/CTUIDs.csv"


total_destinations_df = pd.read_csv(CTUID_source)
area_names_df = pd.read_csv("../../../data/visual_data/CTUID-w-Neighborhood.csv")

# Add 'before' totals

# Add Neighbourhood data
total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    area_names_df,
    base_ctuid_col='CTUID',
    join_ctuid_col='CTUID',
    join_column_name='AREA_NAME',
    new_column_name='Neighbourhood'
)
# display(total_destinations_df.head(20))
before_column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
        'total_column': f'total_destinations_within_{threshold}_mins_before'}

total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    MetricCalculation.calculate_total_destinations(before_csv_path, CTUID_source, before_column_names),
    base_ctuid_col='CTUID',
    join_ctuid_col='CTUID',
    join_column_name=f'total_destinations_within_{threshold}_mins_before',
    new_column_name=f'total_destinations_within_{threshold}_mins_before'
)
# display(total_destinations_df.head(20))
after_column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
        'total_column': f'total_destinations_within_{threshold}_mins_after'}

# Add 'after' totals
total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    MetricCalculation.calculate_total_destinations(after_csv_path, CTUID_source, after_column_names),
    base_ctuid_col='CTUID',
    join_ctuid_col='CTUID',
    join_column_name=f'total_destinations_within_{threshold}_mins_after',
    new_column_name=f'total_destinations_within_{threshold}_mins_after'
)


# Calculate Difference
total_destinations_df[f'total_destinations_within_{threshold}_mins_diff'] = (
    total_destinations_df[f'total_destinations_within_{threshold}_mins_after'] - 
    total_destinations_df[f'total_destinations_within_{threshold}_mins_before']
)

display(total_destinations_df.head(20))

# Save the final DataFrame to CSV
csv_output_path = f'../results/hospitals_{threshold}_before_after_diff.csv'
total_destinations_df.to_csv(csv_output_path, index=False)


,CTUID,Neighbourhood,total_destinations_within_30_mins_before,total_destinations_within_30_mins_after,total_destinations_within_30_mins_diff
0,5350128.04,South Eglinton-Davisville,3,3,0
1,5350363.06,Woburn North,2,2,0
2,5350363.07,Woburn North,2,2,0
3,5350378.23,Agincourt North,0,0,0
4,5350378.24,Agincourt North,0,0,0
5,5350010.01,West Queen West,4,4,0
6,5350010.02,West Queen West,7,7,0
7,5350001.00,South Riverdale,0,0,0
8,5350002.00,St Lawrence-East Bayfront-The Islands,0,0,0
9,5350003.00,South Parkdale,2,2,0


In [14]:
# Travel time to first hospital before and after

before_csv_path = f"../results/first_hospitals_before.csv"
after_csv_path = f"../results/first_hospitals_after.csv"

CTUID_source = "../results/CTUIDs.csv"


total_destinations_df = pd.read_csv(CTUID_source)
area_names_df = pd.read_csv("../../../data/visual_data/CTUID-w-Neighborhood.csv")

before_csv = pd.read_csv(before_csv_path)
after_csv = pd.read_csv(after_csv_path)

# Add 'before' totals

# Add Neighbourhood data
total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    area_names_df,
    base_ctuid_col='CTUID',
    join_ctuid_col='CTUID',
    join_column_name='AREA_NAME',
    new_column_name='Neighbourhood'
)
# # display(total_destinations_df.head(20))
# before_column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
#         'total_column': f'travel_time_to_first_dest_before'}

total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    before_csv,
    base_ctuid_col='CTUID',
    join_ctuid_col='from_id',
    join_column_name=f'travel_time',
    new_column_name=f'travel_time_to_first_dest_before'
)
# display(total_destinations_df.head(20))
# after_column_names = {'from_id': 'from_id','to_id': 'to_id', 'CTUID':'CTUID',
#         'total_column': f'travel_time_to_first_dest_after'}

# Add 'after' totals
total_destinations_df = MetricCalculation.add_column_with_join(
    total_destinations_df,
    after_csv,
    base_ctuid_col='CTUID',
    join_ctuid_col='from_id',
    join_column_name=f'travel_time',
    new_column_name=f'travel_time_to_first_dest_after'
)


# Calculate Difference
total_destinations_df[f'travel_time_to_first_dest_reduction'] = (
    abs(total_destinations_df[f'travel_time_to_first_dest_after'] - 
    total_destinations_df[f'travel_time_to_first_dest_before'])
)

display(total_destinations_df.head(20))

# Save the final DataFrame to CSV
csv_output_path = f'../results/first_hospitals_before_after_reduction.csv'
total_destinations_df.to_csv(csv_output_path, index=False)


,CTUID,Neighbourhood,travel_time_to_first_dest_before,travel_time_to_first_dest_after,travel_time_to_first_dest_reduction
0,5350128.04,South Eglinton-Davisville,22.0,22.0,0.0
1,5350363.06,Woburn North,27.0,27.0,0.0
2,5350363.07,Woburn North,24.0,24.0,0.0
3,5350378.23,Agincourt North,36.0,36.0,0.0
4,5350378.24,Agincourt North,40.0,40.0,0.0
5,5350010.01,West Queen West,8.0,8.0,0.0
6,5350010.02,West Queen West,16.0,16.0,0.0
7,5350001.00,South Riverdale,73.0,73.0,0.0
8,5350002.00,St Lawrence-East Bayfront-The Islands,100.0,100.0,0.0
9,5350003.00,South Parkdale,17.0,17.0,0.0


Pivot the result columns for PowerBI visualization 

In [5]:
MetricCalculation.pivot_totals_to_rows('../results/first_hospitals_before_after_reduction.csv', '../results/first_hospitals_before_after_reduction_pivoted.csv')

Transformed data saved to ../results/first_hospitals_before_after_reduction_pivoted.csv


,CTUID,Neighbourhood,Before_After_Difference,Value
7,5350001.00,South Riverdale,travel_time_to_first_dest_before,73.0
590,5350001.00,South Riverdale,travel_time_to_first_dest_after,73.0
1173,5350001.00,South Riverdale,travel_time_to_first_dest_reduction,0.0
8,5350002.00,St Lawrence-East Bayfront-The Islands,travel_time_to_first_dest_before,100.0
591,5350002.00,St Lawrence-East Bayfront-The Islands,travel_time_to_first_dest_after,100.0
...,...,...,...,...
870,5350378.27,Morningside Heights,travel_time_to_first_dest_after,84.0
1453,5350378.27,Morningside Heights,travel_time_to_first_dest_reduction,0.0
288,5350378.28,Morningside Heights,travel_time_to_first_dest_before,40.0
871,5350378.28,Morningside Heights,travel_time_to_first_dest_after,40.0


In [ ]:
MetricCalculation.group_to_ids_by_ctuid(input_path = "../results/TTM_CT_hospitals_only_before.csv",
output_path = "../results/group.csv")

Grouped data saved to ../results/group.csv


,from_id,to_id_list
0,5350001.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
1,5350002.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
2,5350003.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
3,5350004.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
4,5350005.00,"[54248, 487172, 504810, 515959, 536800, 773891..."
...,...,...
578,5350378.24,"[54248, 487172, 504810, 515959, 536800, 773891..."
579,5350378.25,"[54248, 487172, 504810, 515959, 536800, 773891..."
580,5350378.26,"[54248, 487172, 504810, 515959, 536800, 773891..."
581,5350378.27,"[54248, 487172, 504810, 515959, 536800, 773891..."


### Job Access Calculation

In [3]:
travel_time_matrix_path = "../results/TTM_CT_CT_before.csv"
thresholds = [30]
Employment_data = pd.read_csv("../../../draft/Employment_data.csv")
Job_Metric = MetricCalculation(travel_time_matrix_path)
CT_within_30 = Job_Metric.filter_destinations(threshold = thresholds)
CT_within_30_grouped = MetricCalculation.group_to_ids_by_ctuid(CT_within_30) #.to_csv("../results/CT_within_30_grouped.csv", index=False)
MetricCalculation.sum_jobs_by_from_id_and_fill_missing(CT_within_30_grouped,Employment_data, ctuid_reference_path= "../results/CTUIDs.csv", output_path= "../results/CT_job_access_Before.csv")

Summed job data saved to ../results/CT_job_access_Before.csv


,CTUID,Manufacturing and Warehousing,Retail,Service,Office,Institutional,Community and Entertainment,Grand Total
0,5350003.00,61.0,1402.0,1793.0,2986.0,2852.0,330.0,9424.0
1,5350004.00,1625.0,5576.0,7375.0,20751.0,9202.0,1457.0,45986.0
2,5350005.00,2459.0,7573.0,12264.0,31323.0,10430.0,4003.0,68052.0
3,5350006.00,2139.0,6665.0,10770.0,29988.0,9566.0,2095.0,61223.0
4,5350007.01,1981.0,6355.0,10410.0,29597.0,8772.0,2001.0,59116.0
...,...,...,...,...,...,...,...,...
578,5350001.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,5350002.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580,5350296.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581,5350266.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
travel_time_matrix_path = "../results/TTM_CT_CT_After.csv"
thresholds = [30]
Employment_data = pd.read_csv("../../../draft/Employment_data.csv")
Job_Metric = MetricCalculation(travel_time_matrix_path)
CT_within_30 = Job_Metric.filter_destinations(threshold = thresholds)
CT_within_30_grouped = MetricCalculation.group_to_ids_by_ctuid(CT_within_30) #.to_csv("../results/CT_within_30_grouped.csv", index=False)
MetricCalculation.sum_jobs_by_from_id_and_fill_missing(CT_within_30_grouped,Employment_data, ctuid_reference_path= "../results/CTUIDs.csv", output_path= "../results/CT_job_access_After.csv")

Summed job data saved to ../results/CT_job_access_After.csv


,CTUID,Manufacturing and Warehousing,Retail,Service,Office,Institutional,Community and Entertainment,Grand Total
0,5350003.00,61.0,1402.0,1793.0,2986.0,2852.0,330.0,9424.0
1,5350004.00,1625.0,5576.0,7375.0,20751.0,9202.0,1457.0,45986.0
2,5350005.00,2459.0,7573.0,12264.0,31323.0,10430.0,4003.0,68052.0
3,5350006.00,2139.0,6665.0,10770.0,29988.0,9566.0,2095.0,61223.0
4,5350007.01,2139.0,6543.0,10639.0,29851.0,8970.0,2012.0,60154.0
...,...,...,...,...,...,...,...,...
578,5350001.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,5350002.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580,5350296.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581,5350266.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
print(pd.read_csv("../results/CT_job_access_Before.csv")["Grand Total"].sum())
pd.read_csv("../results/CT_job_access_After.csv")["Grand Total"].sum()

34696661.0


37559166.0

In [ ]:
import pandas as pd

def compute_category_differences(before_csv_path, after_csv_path, output_path=None):
    """
    Computes the 'before', 'after', and absolute difference for each category from the before and after datasets.

    Args:
        before_csv_path (str): Path to the CSV file containing the 'before' data.
        after_csv_path (str): Path to the CSV file containing the 'after' data.
        output_path (str, optional): Path to save the result as a CSV. If None, does not save.

    Returns:
        pd.DataFrame: A DataFrame with columns for 'before', 'after', and 'absolute difference' for each category.
    """
    # Load the before and after datasets
    before_df = pd.read_csv(before_csv_path)
    after_df = pd.read_csv(after_csv_path)

    # Ensure the datasets have the same structure
    if not before_df.columns.equals(after_df.columns):
        raise ValueError("The before and after datasets must have the same columns.")

    # Merge the before and after datasets on the `CTUID` column
    merged_df = before_df.merge(after_df, on="CTUID", suffixes=("_before", "_after"))

    # Extract category columns (excluding `CTUID`)
    categories = [col for col in before_df.columns if col != "CTUID"]

    # Compute the absolute difference for each category
    for category in categories:
        merged_df[f"{category}_absolute_difference"] = merged_df[f"{category}_after"] - merged_df[f"{category}_before"]

    # Save the result to a CSV if output_path is provided
    if output_path:
        merged_df.to_csv(output_path, index=False)
        print(f"Comparison data saved to {output_path}")

    return merged_df

# Example usage:
result_df = compute_category_differences(
    "../results/CT_job_access_Before.csv", "../results/CT_job_access_After.csv", "../results/CT_job_access_Before_After_Diff1.csv"
)


Comparison data saved to ../results/CT_job_access_Before_After_Diff.csv


In [ ]:
result_df = compute_category_differences(
    "../results/CT_job_access_Before.csv", "../results/CT_job_access_After.csv", "../results/CT_job_access_Before_After_Diff2.csv"
)


Pivoted data saved to ../results/CT_job_access_Before_After_Diff2.csv


In [ ]:
import pandas as pd

def v1_compute_category_differences(before_csv_path, after_csv_path, output_path=None):
    """
    Computes the 'before', 'after', and absolute difference for each category from the before and after datasets.

    Args:
        before_csv_path (str): Path to the CSV file containing the 'before' data.
        after_csv_path (str): Path to the CSV file containing the 'after' data.
        output_path (str, optional): Path to save the result as a CSV. If None, does not save.

    Returns:
        pd.DataFrame: A DataFrame with columns for each category in the order: 'before', 'after', and 'absolute difference'.
    """
    # Load the before and after datasets
    before_df = pd.read_csv(before_csv_path)
    after_df = pd.read_csv(after_csv_path)

    # Ensure the datasets have the same structure
    if not before_df.columns.equals(after_df.columns):
        raise ValueError("The before and after datasets must have the same columns.")

    # Merge the before and after datasets on the `CTUID` column
    merged_df = before_df.merge(after_df, on="CTUID", suffixes=("_before", "_after"))

    # Extract category columns (excluding `CTUID`)
    categories = [col for col in before_df.columns if col != "CTUID"]

    # Reorganize the columns: each category with 'before', 'after', and 'difference'
    reordered_columns = ["CTUID"]
    for category in categories:
        merged_df[f"{category}_absolute_difference"] = abs(
            merged_df[f"{category}_after"] - merged_df[f"{category}_before"]
        )
        reordered_columns.extend([f"{category}_before", f"{category}_after", f"{category}_absolute_difference"])

    merged_df = merged_df[reordered_columns]
    merged_df['CTUID'] = merged_df['CTUID'].apply(lambda x: f"{float(x):.2f}")
    # Save the result to a CSV if output_path is provided
    if output_path:
        merged_df.to_csv(output_path, index=False)
        print(f"Comparison data saved to {output_path}")

    return merged_df

# Example usage:
# result_df = compute_category_differences(
#     "CT_job_access_Before.csv", "CT_job_access_After.csv", "comparison_results.csv"
# )


In [ ]:
import pandas as pd

def v2_compute_category_differences(before_csv_path, after_csv_path, output_path=None):
    """
    Computes the 'before', 'after', and absolute difference for each category from the before and after datasets,
    and pivots the job type columns into rows.

    Args:
        before_csv_path (str): Path to the CSV file containing the 'before' data.
        after_csv_path (str): Path to the CSV file containing the 'after' data.
        output_path (str, optional): Path to save the result as a CSV. If None, does not save.

    Returns:
        pd.DataFrame: A pivoted DataFrame with columns: 'CTUID', 'Job_type', 'Time', 'Num_jobs'.
    """
    # Load the before and after datasets
    before_df = pd.read_csv(before_csv_path)
    after_df = pd.read_csv(after_csv_path)

    # Ensure the datasets have the same structure
    if not before_df.columns.equals(after_df.columns):
        raise ValueError("The before and after datasets must have the same columns.")

    # Extract category columns (excluding `CTUID`)
    categories = [col for col in before_df.columns if col != "CTUID"]

    # Calculate the absolute difference for each category
    diff_df = before_df.copy()
    for category in categories:
        diff_df[category] = abs(after_df[category] - before_df[category])

    # Add a time column to each dataset
    before_df = before_df.melt(id_vars=["CTUID"], value_vars=categories, 
                               var_name="Job_type", value_name="Num_jobs")
    before_df["Time"] = "before"

    after_df = after_df.melt(id_vars=["CTUID"], value_vars=categories, 
                             var_name="Job_type", value_name="Num_jobs")
    after_df["Time"] = "after"

    diff_df = diff_df.melt(id_vars=["CTUID"], value_vars=categories, 
                           var_name="Job_type", value_name="Num_jobs")
    diff_df["Time"] = "difference"

    # Concatenate the before, after, and difference data
    combined_df = pd.concat([before_df, after_df, diff_df], ignore_index=True)

    # Reorder the columns
    combined_df = combined_df[["CTUID", "Job_type", "Time", "Num_jobs"]]
    combined_df['CTUID'] = combined_df['CTUID'].apply(lambda x: f"{float(x):.2f}")
    # Save the result to a CSV if output_path is provided
    if output_path:
        combined_df.to_csv(output_path, index=False)
        print(f"Pivoted data saved to {output_path}")

    return combined_df

# Example usage:
# result_df = compute_category_differences(
#     "CT_job_access_Before.csv", "CT_job_access_After.csv", "pivoted_results.csv"
# )


In [38]:
import pandas as pd

def total_compute_category_differences(before_csv_path, after_csv_path, output_path=None):
    """
    Computes the 'before', 'after', and absolute difference for each category from the before and after datasets,
    and pivots the job type columns into rows.

    Args:
        before_csv_path (str): Path to the CSV file containing the 'before' data.
        after_csv_path (str): Path to the CSV file containing the 'after' data.
        output_path (str, optional): Path to save the result as a CSV. If None, does not save.

    Returns:
        pd.DataFrame: A pivoted DataFrame with columns: 'CTUID', 'Job_type', 'Time', 'Num_jobs'.
    """
    # Load the before and after datasets
    before_df = pd.read_csv(before_csv_path)
    after_df = pd.read_csv(after_csv_path)
    before_df = before_df[['CTUID', 'Grand Total']]
    after_df = after_df[['CTUID', 'Grand Total']]
    # Ensure the datasets have the same structure
    if not before_df.columns.equals(after_df.columns):
        raise ValueError("The before and after datasets must have the same columns.")

    # Extract category columns (excluding `CTUID`)
    categories = [col for col in before_df.columns if col != "CTUID"]

    # Calculate the absolute difference for each category
    diff_df = before_df.copy()
    for category in categories:
        diff_df[category] = abs(after_df[category] - before_df[category])

    # Add a time column to each dataset
    before_df = before_df.melt(id_vars=["CTUID"], value_vars=categories, 
                               var_name="Job_type", value_name="Num_jobs")
    before_df["Time"] = "before"

    after_df = after_df.melt(id_vars=["CTUID"], value_vars=categories, 
                             var_name="Job_type", value_name="Num_jobs")
    after_df["Time"] = "after"

    diff_df = diff_df.melt(id_vars=["CTUID"], value_vars=categories, 
                           var_name="Job_type", value_name="Num_jobs")
    diff_df["Time"] = "difference"

    # Concatenate the before, after, and difference data
    combined_df = pd.concat([before_df, after_df, diff_df], ignore_index=True)

    # Reorder the columns
    combined_df = combined_df[["CTUID", "Job_type", "Time", "Num_jobs"]]
    combined_df['CTUID'] = combined_df['CTUID'].apply(lambda x: f"{float(x):.2f}")
    # combined_df = combined_df.sort_values(['CTUID','Time'])
    
    # Save the result to a CSV if output_path is provided
    if output_path:
        combined_df.to_csv(output_path, index=False)
        print(f"Pivoted data saved to {output_path}")

    return combined_df

# Example usage:
# result_df = compute_category_differences(
#     "CT_job_access_Before.csv", "CT_job_access_After.csv", "pivoted_results.csv"
# )


In [ ]:
result_df = total_compute_category_differences(
    "../results/CT_job_access_Before.csv", "../results/CT_job_access_After.csv", "../results/CT_job_access_Before_After_Diff3.csv"
)


Pivoted data saved to ../results/CT_job_access_Before_After_Diff3.csv
